# KAGLE COMPETITION - TITANIC ML from Disaster

### Libraries

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import titanic_utils

### EDA

In [62]:
train = pd.read_csv('data/train.csv', index_col='PassengerId')

print(train.dtypes)

train_clean, replacing_nan_colums = clean_nan_values(train)



train_clean.sort_values(by='Pclass', inplace=True)


train_clean



Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object
Dropped 2 rows due to nan values in Embarked column, corresponding to a 0.224% of the dataset length


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
446,1,1,"Dodge, Master. Washington",male,4.0,0,2,33638,81.8583,A34,S
311,1,1,"Hays, Miss. Margaret Bechstein",female,24.0,0,0,11767,83.1583,C54,C
310,1,1,"Francatelli, Miss. Laura Mabel",female,30.0,0,0,PC 17485,56.9292,E36,C
308,1,1,"Penasco y Castellana, Mrs. Victor de Satode (M...",female,17.0,1,0,PC 17758,108.9000,C65,C
307,1,1,"Fleming, Miss. Margaret",female,NaN,0,0,17421,110.8833,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
379,0,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,NaN,C
380,0,3,"Gustafsson, Mr. Karl Gideon",male,19.0,0,0,347069,7.7750,NaN,S
382,1,3,"Nakid, Miss. Maria (""Mary"")",female,1.0,0,2,2653,15.7417,NaN,C
